<a href="https://colab.research.google.com/github/rpadmanabhan/mdp-rl-project/blob/main/mdp_rl_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install stable-baselines3[extra]

     |████████████████████████████████| 177 kB 7.4 MB/s 
     |████████████████████████████████| 1.5 MB 61.2 MB/s 
     |████████████████████████████████| 1.6 MB 65.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616821 sha256=cef35ba7ca725a0de6e9f4b0dff1f29b3859a7815e3cd48c48442ba2bdc6f8a0
  Stored in directory: /root/.cache/pip/wheels/76/ee/9c/36bfe3e079df99acf5ae57f4e3464ff2771b34447d6d2f2148
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.4.2-py3-none-any.whl size=441027 sha256=3608acba4e15f2f447a62d45c52615291752cc9fe48ddf69c2ab3575ad6adf87
  Stored in directory: /root/.cache/pip/wheels/87/67/2e/6147e7912fe37f5408b80d07527dab807c1d25f5c403a9538a
Successfully built gym AutoROM.accept-rom-license
  Attempting uninstall: gym
    Found existing installation: gym 0.17.3
    Uninstalling gym-0.17.3

In [2]:
from dataclasses import dataclass, field
from typing import Dict, List

import statistics

In [3]:
import gym
from gym import spaces

from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import DQN, PPO, A2C
from stable_baselines3.common.cmd_util import make_vec_env


import numpy as np

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/cmd_util.py:6: FutureWarning: Module ``common.cmd_util`` has been renamed to ``common.env_util`` and will be removed in the future.
  "Module ``common.cmd_util`` has been renamed to ``common.env_util`` and will be removed in the future.", FutureWarning


In [4]:
# some test data
probe_sequences = [
"..."                   
]
ref_sequences = {
    "AY632535.2 Zika virus strain MR 766, complete genome" : "AGTTGTTGATCTGTGTGAGTCAGACTGCGACAGTTCGAGTCTGAAGCGAGAGCTAACAACAGTATCAACAGGTTTAATTTGGATTTGGAAACGAGAGTTTCTGGTCATGAAAAACCCCAAAGAAGAAATCCGGAGGATCCGGATTGTCAATATGCTAAAACGCGGAGTAGCCCGTGTAAACCCCTTGGGAGGTTTGAAGAGGTTGCCAGCCGGACTTCTGCTGGGTCATGGACCCATCAGAATGGTTTTGGCGATACTAGCCTTTTTGAGATTTACAGCAATCAAGCCATCACTGGGCCTTATCAACAGATGGGGTTCCGTGGGGAAAAAAGAGGCTATGGAAATAATAAAGAAGTTCAAGAAAGATCTTGCTGCCATGTTGAGAATAATCAATGCTAGGAAAGAGAGGAAGAGACGTGGCGCAGACACCAGCATCGGAATCATTGGCCTCCTGCTGACTACAGCCATGGCAGCAGAGATCACTAGACGCGGGAGTGCATACTACATGTACTTGGATAGGAGCGATGCCGGGAAGGCCATTTCGTTTGCTACCACATTGGGAGTGAACAAGTGCCACGTACAGATCATGGACCTCGGGCACATGTGTGACGCCACCATGAGTTATGAGTGCCCTATGCTGGATGAGGGAGTGGAACCAGATGATGTCGATTGCTGGTGCAACACGACATCAACTTGGGTTGTGTACGGAACCTGTCATCACAAAAAAGGTGAGGCACGGCGATCTAGAAGAGCCGTGACGCTCCCTTCTCACTCTACAAGGAAGTTGCAAACGCGGTCGCAGACCTGGTTAGAATCAAGAGAATACACGAAGCACTTGATCAAGGTTGAAAACTGGATATTCAGGAACCCCGGGTTTGCGCTAGTGGCCGTTGCCATTGCCTGGCTTTTGGGAAGCTCGACGAGCCAAAAAGTCATATACTTGGTCATGATACTGCTGATTGCCCCGGCATACAGTATCAGGTGCATTGGAGTCAGCAATAGAGACTTCGTGGAGGGCATGTCAGGTGGGACCTGGGTTGATGTTGTCTTGGAACATGGAGGCTGCGTTACCGTGATGGCACAGGACAAGCCAACAGTCGACATAGAGTTGGTCACGACGACGGTTAGTAACATGGCCGAGGTAAGATCCTATTGCTACGAGGCATCGATATCGGACATGGCTTCGGACAGTCGTTGCCCAACACAAGGTGAAGCCTACCTTGACAAGCAATCAGACACTCAATATGTCTGCAAAAGAACATTAGTGGACAGAGGTTGGGGAAACGGTTGTGGACTTTTTGGCAAAGGGAGCTTGGTGACATGTGCCAAGTTTACGTGTTCTAAGAAGATGACCGGGAAGAGCATTCAACCGGAAAATCTGGAGTATCGGATAATGCTATCAGTGCATGGCTCCCAGCATAGCGGGATGATTGGATATGAAACTGACGAAGATAGAGCGAAAGTCGAGGTTACGCCTAATTCACCAAGAGCGGAAGCAACCTTGGGAGGCTTTGGAAGCTTAGGACTTGACTGTGAACCAAGGACAGGCCTTGACTTTTCAGATCTGTATTACCTGACCATGAACAATAAGCATTGGTTGGTGCACAAAGAGTGGTTTCATGACATCCCATTGCCTTGGCATGCTGGGGCAGACACCGGAACTCCACACTGGAACAACAAAGAGGCATTGGTAGAATTCAAGGATGCCCACGCCAAGAGGCAAACCGTCGTCGTTCTGGGGAGCCAGGAAGGAGCCGTTCACACGGCTCTCGCTGGAGCTCTAGAGGCTGAGATGGATGGTGCAAAGGGAAGGCTGTTCTCTGGCCATTTGAAATGCCGCCTAAAAATGGACAAGCTTAGATTGAAGGGCGTGTCATATTCCTTGTGCACTGCGGCATTCACATTCACCAAGGTCCCAGCTGAAACACTGCATGGAACAGTCACAGTGGAGGTGCAGTATGCAGGGACAGATGGACCCTGCAAGATCCCAGTCCAGATGGCGGTGGACATGCAGACCCTGACCCCAGTTGGAAGGCTGATAACCGCCAACCCCGTGATTACTGAAAGCACTGAGAACTCAAAGATGATGTTGGAGCTTGACCCACCATTTGGGGATTCTTACATTGTCATAGGAGTTGGGGACAAGAAAATCACCCACCACTGGCATAGGAGTGGTAGCACCATCGGAAAGGCATTTGAGGCCACTGTGAGAGGCGCCAAGAGAATGGCAGTCCTGGGGGATACAGCCTGGGACTTCGGATCAGTCGGGGGTGTGTTCAACTCACTGGGTAAGGGCATTCACCAGATTTTTGGAGCAGCCTTCAAATCACTGTTTGGAGGAATGTCCTGGTTCTCACAGATCCTCATAGGCACGCTGCTAGTGTGGTTAGGTTTGAACACAAAGAATGGATCTATCTCCCTCACATGCTTGGCCCTGGGGGGAGTGATGATCTTCCTCTCCACGGCTGTTTCTGCTGACGTGGGGTGCTCAGTGGACTTCTCAAAAAAGGAAACGAGATGTGGCACGGGGGTATTCATCTATAATGATGTTGAAGCCTGGAGGGACCGGTACAAGTACCATCCTGACTCCCCCCGCAGATTGGCAGCAGCAGTCAAGCAGGCCTGGGAAGAGGGGATCTGTGGGATCTCATCCGTTTCAAGAATGGAAAACATCATGTGGAAATCAGTAGAAGGGGAGCTCAATGCTATCCTAGAGGAGAATGGAGTTCAACTGACAGTTGTTGTGGGATCTGTAAAAAACCCCATGTGGAGAGGTCCACAAAGATTGCCAGTGCCTGTGAATGAGCTGCCCCATGGCTGGAAAGCCTGGGGGAAATCGTATTTTGTTAGGGCGGCAAAGACCAACAACAGTTTTGTTGTCGACGGTGACACACTGAAGGAATGTCCGCTTGAGCACAGAGCATGGAATAGTTTTCTTGTGGAGGATCACGGGTTTGGAGTCTTCCACACCAGTGTCTGGCTTAAGGTCAGAGAAGATTACTCATTAGAATGTGACCCAGCCGTCATAGGAACAGCTGTTAAGGGAAGGGAGGCCGCGCACAGTGATCTGGGCTATTGGATTGAAAGTGAAAAGAATGACACATGGAGGCTGAAGAGGGCCCACCTGATTGAGATGAAAACATGTGAATGGCCAAAGTCTCACACATTGTGGACAGATGGAGTAGAAGAAAGTGATCTTATCATACCCAAGTCTTTAGCTGGTCCACTCAGCCACCACAACACCAGAGAGGGTTACAGAACCCAAGTGAAAGGGCCATGGCACAGTGAAGAGCTTGAAATCCGGTTTGAGGAATGTCCAGGCACCAAGGTTTACGTGGAGGAGACATGCGGAACTAGAGGACCATCTCTGAGATCAACTACTGCAAGTGGAAGGGTCATTGAGGAATGGTGCTGTAGGGAATGCACAATGCCCCCACTATCGTTTCGAGCAAAAGACGGCTGCTGGTATGGAATGGAGATAAGGCCCAGGAAAGAACCAGAGAGCAACTTAGTGAGGTCAATGGTGACAGCGGGGTCAACCGATCATATGGACCACTTCTCTCTTGGAGTGCTTGTGATTCTACTCATGGTGCAGGAGGGGTTGAAGAAGAGAATGACCACAAAGATCATCATGAGCACATCAATGGCAGTGCTGGTAGTCATGATCTTGGGAGGATTTTCAATGAGTGACCTGGCCAAGCTTGTGATCCTGATGGGTGCTACTTTCGCAGAAATGAACACTGGAGGAGATGTAGCTCACTTGGCATTGGTAGCGGCATTTAAAGTCAGACCAGCCTTGCTGGTCTCCTTCATTTTCAGAGCCAATTGGACACCCCGTGAGAGCATGCTGCTAGCCCTGGCTTCGTGTCTTCTGCAAACTGCGATCTCTGCTCTTGAAGGTGACTTGATGGTCCTCATTAATGGATTTGCTTTGGCCTGGTTGGCAATTCGAGCAATGGCCGTGCCACGCACTGACAACATCGCTCTACCAATCTTGGCTGCTCTAACACCACTAGCTCGAGGCACACTGCTCGTGGCATGGAGAGCGGGCCTGGCTACTTGTGGAGGGATCATGCTCCTCTCCCTGAAAGGGAAAGGTAGTGTGAAGAAGAACCTGCCATTTGTCATGGCCCTGGGATTGACAGCTGTGAGGGTAGTAGACCCTATTAATGTGGTAGGACTACTGTTACTCACAAGGAGTGGGAAGCGGAGCTGGCCCCCTAGTGAAGTTCTCACAGCCGTTGGCCTGATATGTGCACTGGCCGGAGGGTTTGCCAAGGCAGACATTGAGATGGCTGGACCCATGGCTGCAGTAGGCTTGCTAATTGTCAGCTATGTGGTCTCGGGAAAGAGTGTGGACATGTACATTGAAAGAGCAGGTGACATCACATGGGAAAAGGACGCGGAAGTCACTGGAAACAGTCCTCGGCTTGACGTGGCACTGGATGAGAGTGGTGACTTCTCCTTGGTAGAGGAAGATGGTCCACCCATGAGAGAGATCATACTCAAGGTGGTCCTGATGGCCATCTGTGGCATGAACCCAATAGCTATACCTTTTGCTGCAGGAGCGTGGTATGTGTATGTGAAGACTGGGAAAAGGAGTGGCGCCCTCTGGGACGTGCCTGCTCCCAAAGAAGTGAAGAAAGGAGAGACCACAGATGGAGTGTACAGAGTGATGACTCGCAGACTGCTAGGTTCAACACAGGTTGGAGTGGGAGTCATGCAAGAGGGAGTCTTCCACACCATGTGGCACGTTACAAAAGGAGCCGCACTGAGGAGCGGTGAGGGAAGACTTGATCCATACTGGGGGGATGTCAAGCAGGACTTGGTGTCATACTGTGGGCCTTGGAAGTTGGATGCAGCTTGGGATGGACTCAGCGAGGTACAGCTTTTGGCCGTACCTCCCGGAGAGAGGGCCAGAAACATTCAGACCCTGCCTGGAATATTCAAGACAAAGGACGGGGACATCGGAGCAGTTGCTCTGGACTACCCTGCAGGGACCTCAGGATCTCCGATCCTAGACAAATGTGGAAGAGTGATAGGACTCTATGGCAATGGGGTTGTGATCAAGAATGGAAGCTATGTTAGTGCTATAACCCAGGGAAAGAGGGAGGAGGAGACTCCGGTTGAATGTTTCGAACCCTCGATGCTGAAGAAGAAGCAGCTAACTGTCTTGGATCTGCATCCAGGAGCCGGAAAAACCAGGAGAGTTCTTCCTGAAATAGTCCGTGAAGCCATAAAAAAGAGACTCCGGACAGTGATCTTGGCACCAACTAGGGTTGTCGCTGCTGAGATGGAGGAGGCCTTGAGAGGACTTCCGGTGCGTTACATGACAACAGCAGTCAACGTCACCCATTCTGGGACAGAAATCGTTGATTTGATGTGCCATGCCACTTTCACTTCACGCTTACTACAACCCATCAGAGTCCCTAATTACAATCTCAACATCATGGATGAAGCCCACTTCACAGACCCCTCAAGTATAGCTGCAAGAGGATACATATCAACAAGGGTTGAAATGGGCGAGGCGGCTGCCATTTTTATGACTGCCACACCACCAGGAACCCGTGATGCGTTTCCTGACTCTAACTCACCAATCATGGACACAGAAGTGGAAGTCCCAGAGAGAGCCTGGAGCTCAGGCTTTGATTGGGTGACAGACCATTCTGGGAAAACAGTTTGGTTCGTTCCAAGCGTGAGAAACGGAAATGAAATCGCAGCCTGTCTGACAAAGGCTGGAAAGCGGGTCATACAGCTCAGCAGGAAGACTTTTGAGACAGAATTTCAGAAAACAAAAAATCAAGAGTGGGACTTTGTCATAACAACTGACATCTCAGAGATGGGCGCCAACTTCAAGGCTGACCGGGTCATAGACTCTAGGAGATGCCTAAAACCAGTCATACTTGATGGTGAGAGAGTCATCTTGGCTGGGCCCATGCCTGTCACGCATGCTAGTGCTGCTCAGAGGAGAGGACGTATAGGCAGGAACCCTAACAAACCTGGAGATGAGTACATGTATGGAGGTGGGTGTGCAGAGACTGATGAAGGCCATGCACACTGGCTTGAAGCAAGAATGCTTCTTGACAACATCTACCTCCAGGATGGCCTCATAGCCTCGCTCTATCGGCCTGAGGCCGATAAGGTAGCCGCCATTGAGGGAGAGTTTAAGCTGAGGACAGAGCAAAGGAAGACCTTCGTGGAACTCATGAAGAGAGGAGACCTTCCCGTCTGGCTAGCCTATCAGGTTGCATCTGCCGGAATAACTTACACAGACAGAAGATGGTGCTTTGATGGCACAACCAACAACACCATAATGGAAGACAGTGTACCAGCAGAGGTTTGGACAAAGTATGGAGAGAAGAGAGTGCTCAAACCGAGATGGATGGATGCTAGGGTCTGTTCAGACCATGCGGCCCTGAAGTCGTTCAAAGAATTCGCCGCTGGAAAAAGAGGAGCGGCTTTGGGAGTAATGGAGGCCCTGGGAACACTGCCAGGACACATGACAGAGAGGTTTCAGGAAGCCATTGACAACCTCGCCGTGCTCATGCGAGCAGAGACTGGAAGCAGGCCTTATAAGGCAGCGGCAGCCCAACTGCCGGAGACCCTAGAGACCATTATGCTCTTAGGTTTGCTGGGAACAGTTTCACTGGGGATCTTCTTCGTCTTGATGCGGAATAAGGGCATCGGGAAGATGGGCTTTGGAATGGTAACCCTTGGGGCCAGTGCATGGCTCATGTGGCTTTCGGAAATTGAACCAGCCAGAATTGCATGTGTCCTCATTGTTGTGTTTTTATTACTGGTGGTGCTCATACCCGAGCCAGAGAAGCAAAGATCTCCCCAAGATAACCAGATGGCAATTATCATCATGGTGGCAGTGGGCCTTCTAGGTTTGATAACTGCAAACGAACTTGGATGGCTGGAAAGAACAAAAAATGACATAGCTCATCTAATGGGAAGGAGAGAAGAAGGAGCAACCATGGGATTCTCAATGGACATTGATCTGCGGCCAGCCTCCGCCTGGGCTATCTATGCCGCATTGACAACTCTCATCACCCCAGCTGTCCAACATGCGGTAACCACTTCATACAACAACTACTCCTTAATGGCGATGGCCACACAAGCTGGAGTGCTGTTTGGCATGGGCAAAGGGATGCCATTTATGCATGGGGACCTTGGAGTCCCGCTGCTAATGATGGGTTGCTATTCACAATTAACACCCCTGACTCTGATAGTAGCTATCATTCTGCTTGTGGCGCACTACATGTACTTGATCCCAGGCCTACAAGCGGCAGCAGCGCGTGCTGCCCAGAAAAGGACAGCAGCTGGCATCATGAAGAATCCCGTTGTGGATGGAATAGTGGTAACTGACATTGACACAATGACAATAGACCCCCAGGTGGAGAAGAAGATGGGACAAGTGTTACTCATAGCAGTAGCCATCTCCAGTGCTGTGCTGCTGCGGACCGCCTGGGGATGGGGGGAGGCTGGAGCTCTGATCACAGCAGCGACCTCCACCTTGTGGGAAGGCTCTCCAAACAAATACTGGAACTCCTCTACAGCCACCTCACTGTGCAACATCTTCAGAGGAAGCTATCTGGCAGGAGCTTCCCTTATCTATACAGTGACGAGAAACGCTGGCCTGGTTAAGAGACGTGGAGGTGGGACGGGAGAGACTCTGGGAGAGAAGTGGAAAGCTCGTCTGAATCAGATGTCGGCCCTGGAGTTCTACTCTTATAAAAAGTCAGGTATCACTGAAGTGTGTAGAGAGGAGGCTCGCCGTGCCCTCAAGGATGGAGTGGCCACAGGAGGACATGCCGTATCCCGGGGAAGTGCAAAGATCAGATGGTTGGAGGAGAGAGGATATCTGCAGCCCTATGGGAAGGTTGTTGACCTCGGATGTGGCAGAGGGGGCTGGAGCTATTATGCCGCCACCATCCGCAAAGTGCAGGAGGTGAGAGGATACACAAAGGGAGGTCCCGGTCATGAAGAACCCATGCTGGTGCAAAGCTATGGGTGGAACATAGTTCGTCTCAAGAGTGGAGTGGACGTCTTCCACATGGCGGCTGAGCCGTGTGACACTCTGCTGTGTGACATAGGTGAGTCATCATCTAGTCCTGAAGTGGAAGAGACACGAACACTCAGAGTGCTCTCTATGGTGGGGGACTGGCTTGAAAAAAGACCAGGGGCCTTCTGTATAAAGGTGCTGTGCCCATACACCAGCACTATGATGGAAACCATGGAGCGACTGCAACGTAGGCATGGGGGAGGATTAGTCAGAGTGCCATTGTGTCGCAACTCCACACATGAGATGTACTGGGTCTCTGGGGCAAAGAGCAACATCATAAAAAGTGTGTCCACCACAAGTCAGCTCCTCCTGGGACGCATGGATGGCCCCAGGAGGCCAGTGAAATATGAGGAGGATGTGAACCTCGGCTCGGGTACACGAGCTGTGGCAAGCTGTGCTGAGGCTCCTAACATGAAAATCATCGGCAGGCGCATTGAGAGAATCCGCAATGAACATGCAGAAACATGGTTTCTTGATGAAAACCACCCATACAGGACATGGGCCTACCATGGGAGCTACGAAGCCCCCACGCAAGGATCAGCGTCTTCCCTCGTGAACGGGGTTGTTAGACTCCTGTCAAAGCCTTGGGACGTGGTGACTGGAGTTACAGGAATAGCCATGACTGACACCACACCATACGGCCAACAAAGAGTCTTCAAAGAAAAAGTGGACACCAGGGTGCCAGATCCCCAAGAAGGCACTCGCCAGGTAATGAACATAGTCTCTTCCTGGCTGTGGAAGGAGCTGGGGAAACGCAAGCGGCCACGCGTCTGCACCAAAGAAGAGTTTATCAACAAGGTGCGCAGCAATGCAGCACTGGGAGCAATATTTGAAGAGGAAAAAGAATGGAAGACGGCTGTGGAAGCTGTGAATGATCCAAGGTTTTGGGCCCTAGTGGATAGGGAGAGAGAACACCACCTGAGAGGAGAGTGTCACAGCTGTGTGTACAACATGATGGGAAAAAGAGAAAAGAAGCAAGGAGAGTTCGGGAAAGCAAAAGGTAGCCGCGCCATCTGGTACATGTGGTTGGGAGCCAGATTCTTGGAGTTTGAAGCCCTTGGATTCTTGAACGAGGACCATTGGATGGGAAGAGAAAACTCAGGAGGTGGAGTCGAAGGGTTAGGATTGCAAAGACTTGGATACATTCTAGAAGAAATGAATCGGGCACCAGGAGGAAAGATGTACGCAGATGACACTGCTGGCTGGGACACCCGCATTAGTAAGTTTGATCTGGAGAATGAAGCTCTGATTACCAACCAAATGGAGGAAGGGCACAGAACTCTGGCGTTGGCCGTGATTAAATACACATACCAAAACAAAGTGGTGAAGGTTCTCAGACCAGCTGAAGGAGGAAAAACAGTTATGGACATCATTTCAAGACAAGACCAGAGAGGGAGTGGACAAGTTGTCACTTATGCTCTCAACACATTCACCAACTTGGTGGTGCAGCTTATCCGGAACATGGAAGCTGAGGAAGTGTTAGAGATGCAAGACTTATGGTTGTTGAGGAAGCCAGAGAAAGTGACCAGATGGTTGCAGAGCAATGGATGGGATAGACTCAAACGAATGGCGGTCAGTGGAGATGACTGCGTTGTGAAGCCAATCGATGATAGGTTTGCACATGCCCTCAGGTTCTTGAATGACATGGGAAAAGTTAGGAAAGACACACAGGAGTGGAAACCCTCGACTGGATGGAGCAATTGGGAAGAAGTCCCGTTCTGCTCCCACCACTTCAACAAGCTGTACCTCAAGGATGGGAGATCCATTGTGGTCCCTTGCCGCCACCAAGATGAACTGATTGGCCGAGCTCGCGTCTCACCAGGGGCAGGATGGAGCATCCGGGAGACTGCCTGTCTTGCAAAATCATATGCGCAGATGTGGCAGCTCCTTTATTTCCACAGAAGAGACCTTCGACTGATGGCTAATGCCATTTGCTCGGCTGTGCCAGTTGACTGGGTACCAACTGGGAGAACCACCTGGTCAATCCATGGAAAGGGAGAATGGATGACCACTGAGGACATGCTCATGGTGTGGAATAGAGTGTGGATTGAGGAGAACGACCATATGGAGGACAAGACTCCTGTAACAAAATGGACAGACATTCCCTATCTAGGAAAAAGGGAGGACTTATGGTGTGGATCCCTTATAGGGCACAGACCCCGCACCACTTGGGCTGAAAACATCAAAGACACAGTCAACATGGTGCGCAGGATCATAGGTGATGAAGAAAAGTACATGGACTATCTATCCACCCAAGTCCGCTACTTGGGTGAGGAAGGGTCCACACCCGGAGTGTTGTAAGCACCAATTTTAGTGTTGTCAGGCCTGCTAGTCAGCCACAGTTTGGGGAAAGCTGTGCAGCCTGTAACCCCCCCAGGAGAAGCTGGGAAACCAAGCTCATAGTCAGGCCGAGAACGCCATGGCACGGAAGAAGCCATGCTGCCTGTGAGCCCCTCAGAGGACACTGAGTCAAAAAACCCCACGCGCTTGGAAGCGCAGGATGGGAAAAGAAGGTGGCGACCTTCCCCACCCTTCAATCTGGGGCCTGAACTGGAGACTAGCTGTGAATCTCCAGCAGAGGGACTAGTGGTTAGAGGAGACCCCCCGGAAAACGCAAAACAGCATATTGACGTGGGAAAGACCAGAGACTCCATGAGTTTCCACCACGCTGGCCGCCAGGCACAGATCGCCGAACTTCGGCGGCCGGTGTGGGGAAATCCATGGTTTCT"
}
print("Number of initial Probe candidates: ", len(probe_sequences))
print("Reference Sequence Length(s): ", [(seq_name, len(seq)) for seq_name, seq in ref_sequences.items()])

Number of initial Probe candidates:  215
Reference Sequence Length(s):  [('AY632535.2 Zika virus strain MR 766, complete genome', 10794)]


In [6]:
@dataclass(frozen = True)
class ProbeOligo:
    seq: str
    hyb_extension: int = 50

    def ret_coverage_range(self, ref_seq):
        start = ref_seq.find(self.seq)
        assert start != -1
        # return coverage [start, end) range in reference sequence
        return start - self.hyb_extension, start + len(self.seq) + self.hyb_extension

@dataclass()
class RefSeqGroup:
    ref_sequences: Dict[str, str]

    def __post_init__(self):
        self.coverage_profile = {seq_name: np.zeros(len(seq), dtype = np.int16) for seq_name, seq in self.ref_sequences.items()}

    def _reset_coverage_profile(self):
        """
        """
        self.coverage_profile = {seq_name: np.zeros(len(seq), dtype = np.int16) for seq_name, seq in self.ref_sequences.items()}

    def update_coverage_profile(self, probe: ProbeOligo):
        """ Update base level coverage for all reference sequences
        """
        for seq_name,seq in self.ref_sequences.items():
            start, end = probe.ret_coverage_range(seq)
            # need some bounds because of hyb_extension param for a ProbeOligo
            start = max(0, start)
            end = min(len(seq), end)
            for idx in range(start, end):
                self.coverage_profile[seq_name][idx] += 1

    def ret_num_bases_covered(self):
        """ Provide a count for the number of bases covered for each reference sequence
        """
        num_bases_covered = []
        for seq_name, seq in self.ref_sequences.items():
            num_bases_covered.append(np.count_nonzero(self.coverage_profile[seq_name]))
        return num_bases_covered

    def ret_mean_bases_covered(self):
        """Return mean bases covered across all reference sequences
        """
        return statistics.mean(self.ret_num_bases_covered())


class OligoEnv(gym.Env):
    def __init__(self, *args, ref_seq_group: RefSeqGroup, oligos: List[ProbeOligo], **kwargs):
        super(OligoEnv, self).__init__()

        # initialize group of ref sequences
        self.ref_seq_group = ref_seq_group
        self.total_ref_seq_len = sum([len(seq) for seq_name, seq in self.ref_seq_group.ref_sequences.items()])

        # threshold for coverage across reference sequence group
        self.coverage_threshold = kwargs.get("coverage_threshold", 1.0)

        # list of candidate oligos we have
        self.oligos = oligos

        # where is the agent now ?
        self.chosen_oligos = []

        # action space
        self.action_space = spaces.Discrete(len(self.oligos))

        # observation space - state of the agent
        self.observation_space = spaces.MultiBinary(len(self.oligos))

        # some reward related constants
        self._reward_oligo_penalty = -1
        self._reward_action_masking = -50

        # variables to reset for state space
        self._actions_taken = None

    def _check_termination(self, observation):
        # terminate if we hit the desired coverage threshold
        return self.ref_seq_group.ret_mean_bases_covered() / self.total_ref_seq_len >= self.coverage_threshold

    def _compute_reward(self, action):
        oligo = self.oligos[action]
        cur_bases_covered = self.ref_seq_group.ret_mean_bases_covered()
        # update coverage profile
        self.ref_seq_group.update_coverage_profile(self.oligos[action])
        now_bases_covered = self.ref_seq_group.ret_mean_bases_covered()
        # delayed reward when we end, for now just penalize this oligo
        return self._reward_oligo_penalty + now_bases_covered - cur_bases_covered

    def step(self, action):
        """
        """
        # we do not want to repeat an oligo so we want to have some mechanism for penalizing this
        reward = self._reward_action_masking
        if action not in self._actions_taken:
            # keep track of oligos added
            self._actions_taken.add(action)
            # compute reward
            reward = self._compute_reward(action)

        # which oligos have been picked
        observation = np.array([1 if idx in self._actions_taken else 0 for idx in range(0, len(self.oligos))], dtype = np.int8)

        # have we met stopping criteria
        done = bool(self._check_termination(observation))

        # we accumulate delayed reward at the end
        #if done:
        #    reward = self.ref_seq_group.ret_mean_bases_covered()

        # not much info to add currently
        info = {}

        return observation, reward, done, info

    def reset(self):

        # no oligos picked
        self._actions_taken = set({})
        init_observation = np.zeros(len(self.oligos), dtype = np.int8)
        self.ref_seq_group._reset_coverage_profile()

        return init_observation

    def render(self, mode = "console"):
        # To Do: Add some neat visualization of the oligo bindings to reference
        pass

    def close (self):
        pass

In [7]:
# init Probe and Reference datastructures
oligos = [ProbeOligo(probe_seq) for probe_seq in probe_sequences]
ref_seq_group = RefSeqGroup(ref_sequences)

In [8]:
# init custom environment
env = OligoEnv(ref_seq_group=ref_seq_group, oligos = oligos)

In [9]:
# check that the environment works
check_env(env)


In [10]:
# learn an optimal policy
env_wrap = make_vec_env(lambda: env, n_envs = 1)


In [16]:
model = A2C("MlpPolicy", env, verbose = 1).learn(10000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 236      |
|    ep_rew_mean        | 6.25e+03 |
| time/                 |          |
|    fps                | 652      |
|    iterations         | 100      |
|    time_elapsed       | 0        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -5.37    |
|    explained_variance | 0.00784  |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 2.74e+03 |
|    value_loss         | 3.07e+05 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 304      |
|    ep_rew_mean        | 3.42e+03 |
| time/                 |          |
|    fps                | 693      |
|    iterations         | 200      |
|    time_elapsed 

In [30]:
# see how it performs
obs = env.reset()
n_steps = 1000
actions = set({})
for step in range(n_steps):
    action, _ = model.predict(obs, deterministic=False)
    print(step, action)
    observation, reward, done, info = env.step(action)
    actions.add(action)
    if done:
      break


0 70
1 27
2 76
3 4
4 108
5 33
6 209
7 138
8 174
9 34
10 56
11 76
12 104
13 41
14 79
15 144
16 103
17 18
18 188
19 140
20 11
21 176
22 121
23 56
24 24
25 82
26 131
27 95
28 140
29 71
30 109
31 169
32 127
33 117
34 64
35 80
36 9
37 51
38 149
39 82
40 71
41 22
42 176
43 44
44 189
45 47
46 32
47 27
48 130
49 99
50 7
51 50
52 192
53 89
54 89
55 147
56 26
57 108
58 178
59 209
60 16
61 62
62 184
63 71
64 14
65 172
66 48
67 83
68 141
69 130
70 92
71 104
72 125
73 64
74 156
75 94
76 36
77 114
78 126
79 29
80 0
81 110
82 54
83 169
84 6
85 110
86 98
87 14
88 15
89 29
90 88
91 102
92 80
93 73
94 114
95 59
96 201
97 155
98 26
99 33
100 29
101 2
102 182
103 29
104 25
105 43
106 178
107 176
108 82
109 137
110 10
111 16
112 96
113 137
114 212
115 33
116 7
117 83
118 203
119 55
120 98
121 178
122 180
123 182
124 81
125 206
126 198
127 154
128 79
129 20
130 31
131 78
132 151
133 134
134 39
135 101
136 174
137 149
138 181
139 85
140 176
141 116
142 18
143 150
144 35
145 105
146 68
147 170
148 105
149 21


In [31]:
print(actions)
print(len(actions))

{0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41, 42, 43, 44, 45, 47, 48, 50, 51, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 70, 71, 72, 73, 75, 76, 78, 79, 80, 81, 82, 83, 85, 86, 87, 88, 89, 90, 91, 92, 94, 95, 96, 97, 98, 99, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 113, 114, 116, 117, 118, 119, 121, 125, 126, 127, 128, 130, 131, 132, 133, 134, 136, 137, 138, 139, 140, 141, 143, 144, 145, 147, 149, 150, 151, 152, 153, 154, 155, 156, 157, 160, 161, 162, 163, 167, 169, 170, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 186, 187, 188, 189, 190, 192, 193, 196, 197, 198, 199, 200, 201, 203, 204, 206, 207, 209, 212, 214}
172


In [22]:
env.ref_seq_group.coverage_profile

{'AY632535.2 Zika virus strain MR 766, complete genome': array([1, 1, 1, ..., 0, 0, 0], dtype=int16)}

In [250]:
test_oligos = ["AGTTGTTGATCTGTGTGAGTCAG", "TGTGAGTCAGACTG", "CTGCGACAGTTCGAGTCTGAAGCGAGAGCTAACAAC", "AGTTGTTGATCTGTGTGAGTCAGACTGC", "GACAGTTCGAGTCTGAAGCGAGAGCTAACAA", "CAGTATCAACAGGTTTAATTTGGATTTGGAAACGAGAGTTTCTGGTCATGAAAAA"]
test_seqs = {"test_seq": "AGTTGTTGATCTGTGTGAGTCAGACTGCGACAGTTCGAGTCTGAAGCGAGAGCTAACAACAGTATCAACAGGTTTAATTTGGATTTGGAAACGAGAGTTTCTGGTCATGAAAAA"}

In [253]:
# init Probe and Reference datastructures
oligos = [ProbeOligo(probe_seq, hyb_extension=0) for probe_seq in test_oligos]
ref_seq_group = RefSeqGroup(test_seqs)

In [255]:
# init custom environment
env = OligoEnv(ref_seq_group=ref_seq_group, oligos = oligos)
check_env(env)
model = A2C("MlpPolicy", env, verbose = 1).learn(10000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 6.37     |
|    ep_rew_mean        | 4.6      |
| time/                 |          |
|    fps                | 569      |
|    iterations         | 100      |
|    time_elapsed       | 0        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -1.71    |
|    explained_variance | 0.0635   |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -137     |
|    value_loss         | 6.44e+03 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 4.7      |
|    ep_rew_mean        | 75       |
| time/                 |          |
|    fps                | 588      |
|    iterations         | 200      |
|    time_elapsed 

In [256]:
# see how it performs
obs = env.reset()
n_steps = 100
actions = set({})
for step in range(n_steps):
    action, other_ = model.predict(obs, deterministic=True)
    print(step, action, other_)
    out = env.step(action)
    print(out)
    actions.add(action)

0 5 None
(array([0, 0, 0, 0, 0, 1], dtype=int8), 54, False, {})
1 5 None
(array([0, 0, 0, 0, 0, 1], dtype=int8), -50, False, {})
2 5 None
(array([0, 0, 0, 0, 0, 1], dtype=int8), -50, False, {})
3 5 None
(array([0, 0, 0, 0, 0, 1], dtype=int8), -50, False, {})
4 5 None
(array([0, 0, 0, 0, 0, 1], dtype=int8), -50, False, {})
5 5 None
(array([0, 0, 0, 0, 0, 1], dtype=int8), -50, False, {})
6 5 None
(array([0, 0, 0, 0, 0, 1], dtype=int8), -50, False, {})
7 5 None
(array([0, 0, 0, 0, 0, 1], dtype=int8), -50, False, {})
8 5 None
(array([0, 0, 0, 0, 0, 1], dtype=int8), -50, False, {})
9 5 None
(array([0, 0, 0, 0, 0, 1], dtype=int8), -50, False, {})
10 5 None
(array([0, 0, 0, 0, 0, 1], dtype=int8), -50, False, {})
11 5 None
(array([0, 0, 0, 0, 0, 1], dtype=int8), -50, False, {})
12 5 None
(array([0, 0, 0, 0, 0, 1], dtype=int8), -50, False, {})
13 5 None
(array([0, 0, 0, 0, 0, 1], dtype=int8), -50, False, {})
14 5 None
(array([0, 0, 0, 0, 0, 1], dtype=int8), -50, False, {})
15 5 None
(array([0, 